In [3]:
import numpy as np 
import pandas as pd 

In [4]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [1]:
import codecs
batch_size = 64  
epochs = 50 
latent_dim = 256  
num_samples = 10000  
data_path = 'eng_-french.csv'

In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [5]:
data=pd.read_csv(data_path)
data.head()
print(data.columns)

FileNotFoundError: ignored

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4918730032125215450
]


In [ ]:
input_texts=data['English words/sentences']
target_texts=data['French words/sentences']
input_texts=input_texts[:75000]
target_texts=target_texts[:75000]
target_texts=['\t'+text+'\n' for text in target_texts]

In [ ]:
print(input_texts.shape)

(75000,)


In [ ]:
for text in input_texts:
    for char in text:
        input_characters.add(char)
input_characters
for text in target_texts:
    for char in text:
        target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 75000
Number of unique input tokens: 79
Number of unique output tokens: 105
Max sequence length for inputs: 27
Max sequence length for outputs: 74


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

In [ ]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.            
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.    
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/50
938/938 [==============================] - 19s 20ms/step - loss: 0.7825 - accuracy: 0.7831 - val_loss: 0.7593 - val_accuracy: 0.7727
Epoch 2/50
938/938 [==============================] - 16s 17ms/step - loss: 0.5084 - accuracy: 0.8482 - val_loss: 0.6012 - val_accuracy: 0.8190
Epoch 3/50
938/938 [==============================] - 16s 18ms/step - loss: 0.4199 - accuracy: 0.8738 - val_loss: 0.5255 - val_accuracy: 0.8416
Epoch 4/50
938/938 [==============================] - 17s 18ms/step - loss: 0.3720 - accuracy: 0.8878 - val_loss: 0.4856 - val_accuracy: 0.8538
Epoch 5/50
938/938 [==============================] - 17s 18ms/step - loss: 0.3416 - accuracy: 0.8963 - val_loss: 0.4581 - val_accuracy: 0.8611
Epoch 6/50
938/938 [==============================] - 16s 17ms/step - loss: 0.3202 - accuracy: 0.9023 - val_loss: 0.4366 - val_accuracy: 0.8675
Epoch 7/50
938/938 [==============================] - 16s 17ms/step - loss: 0.3040 - accuracy: 0.9071 - val_loss: 0.4246 - val_accuracy:

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Cours ça !

-
Input sentence: Run!
Decoded sentence: Cours ça !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ma compassiffe.

-
Input sentence: Fire!
Decoded sentence: La femme a cassée.

-
Input sentence: Help!
Decoded sentence: À l'entende !

-
Input sentence: Jump.
Decoded sentence: Allez en voulute !

-
Input sentence: Stop!
Decoded sentence: Asseyez-le !

-
Input sentence: Stop!
Decoded sentence: Asseyez-le !

-
Input sentence: Stop!
Decoded sentence: Asseyez-le !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Va voir.

-
Input sentence: Go on.
Decoded sentence: Va voir.

-
Input sentence: Go on.
Decoded sentence: Va voir.

-
Input sentence: Hello!
Decoded sentence: Salut cette fille !

-
Input sentence: Hello!
Decoded sentence: Salut cette fille

In [ ]:
model.save('model.h5')
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')